# Nova 멀티모달 이해 프롬프트 엔지니어링 샌드박스

효과적인 시스템 프롬프트와 도구 사용 구성을 설계하는 데는 종종 시간이 걸립니다. 복잡한 도구 사용 JSON 구조의 경우, 비디오 이해 도구와 통합하기 전에 테스트하는 것이 모범 사례입니다.

이 노트북을 사용하면 바로 그것을 할 수 있습니다. 단일 이미지나 짧은 비디오와 같은 가벼운 입력으로 프롬프트와 구성을 테스트하여 전체 비디오를 처리하지 않고도 결과를 빠르게 검증할 수 있어 반복 작업이 더 빠르고 효율적입니다.

In [ ]:
import boto3
import json
import time
from utils import bedrock_tool
from IPython.display import JSON, Markdown, display

## Nova 이미지 이해 구성 테스트

다음 샘플은 좌표와 함께 객체를 반환하는 시스템 프롬프트와 toolUse 구성을 정의하는 방법을 보여줍니다.

In [ ]:
prompt = "이미지에서 객체를 감지하고 좌표를 반환하세요"

In [ ]:
tool_config ={
    "toolChoice": {
        "tool": {
            "name": "tool_result_with_bb"
        }
    },
    "tools": [
        {
            "toolSpec": {
                "name": "tool_result_with_bb",
                "description": "이미지에서 객체를 감지하고 좌표를 반환합니다",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "objects": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "name": {
                                            "type": "string",
                                            "enum": [
                                                "person",
                                                "car",
                                                "package"
                                            ]
                                        },
                                        "coordinates": {
                                            "type": "object",
                                            "properties": {
                                                "x": {
                                                    "type": "number"
                                                },
                                                "width": {
                                                    "type": "number"
                                                },
                                                "y": {
                                                    "type": "number"
                                                },
                                                "height": {
                                                    "type": "number"
                                                }
                                            },
                                            "required": [
                                                "x",
                                                "y",
                                                "width",
                                                "height"
                                            ]
                                        }
                                    },
                                    "required": [
                                        "name",
                                        "coordinates"
                                    ]
                                }
                            }
                        },
                        "required": [
                            "objects"
                        ]
                    }
                }
            }
        }
    ]
}

로컬 이미지를 사용하여 프롬프트를 테스트합니다.

In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id='amazon.nova-lite-v1:0',
    prompt=prompt, 
    tool_config=tool_config, 
    inference_config={
                "maxTokens": 500,
                "topP": 0.1,
                "temperature": 0.7
              },
    local_file_path="./samples/door-bell-camera.jpeg"
)
custom_output = bedrock_tool.parse_converse_response(response)
display(JSON(custom_output))

## 비디오 분석
다음은 시작점으로 사용할 수 있는 비디오 이해 샘플입니다.

In [ ]:
prompt = "당신은 비디오를 검토하여 안전하지 않은 콘텐츠를 감지하는 콘텐츠 조정 전문가입니다. 결과에 사고 과정을 포함하지 마세요."

In [ ]:
tool_config = {
      "tools": [
        {
          "toolSpec": {
            "name": "tool_result",
            "description": "제공된 비디오에 안전하지 않은 콘텐츠가 포함되어 있는지 감지하고 다음 범주로 분류합니다.",
            "inputSchema": {
              "json": {
                "type": "object",
                "properties": {
                  "suggestive": {
                    "type": "boolean",
                    "description": "비디오에 상반신 노출 남성, 노출이 심한 의상 또는 기타 부적절한 장면이 포함되어 있는지 나타냅니다."
                  },
                  "tobacco": {
                    "type": "boolean",
                    "description": "비디오에 담배 또는 흡연과 관련된 장면이 포함되어 있는지 나타냅니다."
                  },
                  "alcohol": {
                    "type": "boolean",
                    "description": "비디오에 알코올과 관련된 장면이 포함되어 있는지 나타냅니다."
                  }
                },
                "required": [
                  "suggestive",
                  "tobacco",
                  "alcohol"
                ]
              }
            }
          }
        }
      ]
    }


In [ ]:
response = bedrock_tool.bedrock_converse(
    model_id='global.amazon.nova-2-lite-v1:0',
    prompt=prompt, 
    tool_config=tool_config, 
    inference_config={
                "maxTokens": 500,
                "topP": 0.1,
                "temperature": 0.7
              },
    local_file_path="./samples/content-moderation-demo.mp4"
)
custom_output = bedrock_tool.parse_converse_response(response)
display(JSON(custom_output))